In [5]:
import os
import requests
from dotenv import load_dotenv

In [12]:
import anthropic

# Load environment variables from .env file
load_dotenv()

client = anthropic.Anthropic(
    api_key=os.getenv('ANTHROPIC_API_KEY')
)

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)


[TextBlock(text="Tears of ancient earth,\nMinerals dissolved in time,\nRivers carry salt to sea -\nNature's briny paradigm.\n\nEons pass, waves roll on,\nEach drop holds stories untold,\nOf rocks worn down to grains,\nMaking waters brackish, bold.", type='text')]


In [22]:
from flask import Flask, jsonify, request
from pydantic import BaseModel, ValidationError

class Payload(BaseModel):
    message: str
    

app = Flask(__name__)

@app.route('/', methods=['POST'])
def get_message():
    try:
        payload = Payload(**request.json)
        message = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1000,
            temperature=0,
            system="You are a world-class poet. Respond only with short poems.",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": payload.message
                        }
                    ]
                }
            ]
        )

        # response = {
        #     'reply': message.content
        # }

        # return jsonify({'hello':'world'}), 200
        # return jsonify(response), 200
        return jsonify(message.content[0].to_json()), 200
    except ValidationError as e:
        return jsonify(e.errors()), 400

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Oct/2024 18:11:39] "POST / HTTP/1.1" 200 -


In [21]:
import json
json.dumps(message.content[0].to_dict(), indent=4)

'{\n    "text": "Tears of ancient earth,\\nMinerals dissolved in time,\\nRivers carry salt to sea -\\nNature\'s briny paradigm.\\n\\nEons pass, waves roll on,\\nEach drop holds stories untold,\\nOf rocks worn down to grains,\\nMaking waters brackish, bold.",\n    "type": "text"\n}'